---


<div style="background-color: #1e1e1e; color: #cfd8dc; padding: 25px; border-radius: 15px; border: 1px solid #37474f; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; box-shadow: 0 10px 20px rgba(0,0,0,0.5);">
    
<div style="text-align: center; border-bottom: 2px solid #7c4dff; padding-bottom: 20px; margin-bottom: 30px;">
        <h1 style="color: #7c4dff; font-weight: 800; margin-bottom: 5px;">
             Dynamic Hashing & Pruning (DHP)
        </h1>
        <p style="font-size: 1.1em; color: #b0bec5; letter-spacing: 1px;">
            Optimizing the <span style="color: #ff4081; font-weight: bold;">$C_2$</span> Bottleneck
        </p>
    </div>

<h2 style="color: #00e5ff; border-left: 4px solid #00e5ff; padding-left: 15px;">1. The Bottleneck: Why optimize $C_2$?</h2>
    <p>
        In standard Apriori, the most expensive step is generating the candidate pairs ($C_2$). Even if you filter out infrequent individual items, the number of possible pairs explodes combinatorially.
    </p>
    <div style="background-color: #263238; padding: 15px; border-radius: 8px; text-align: center; border: 1px dashed #546e7a;">
        <p style="margin: 0; color: #eceff1;">
            If you have 1,000 frequent items, you must check:<br>
            $$ \frac{1000 \times 999}{2} \approx 500,000 \text{ pairs} $$
        </p>
    </div>

 <h2 style="color: #00e5ff; margin-top: 40px; border-left: 4px solid #00e5ff; padding-left: 15px;">2. The Hashing Strategy</h2>
    <p>
        The core idea is to <b>"peek ahead"</b>. While we scan the database to count individual items ($L_1$), we can simultaneously gather rough data about pairs using a <b>Hash Table</b>.
    </p>

<ul style="list-style-type: none; padding: 0;">
        <li style="margin-bottom: 15px; display: flex; align-items: start;">
            <span style="color: #7c4dff; font-weight: bold; margin-right: 10px; font-size: 1.2em;">1.</span>
            <div><b>Scan:</b> As we read a transaction, generate all possible 2-item subsets.</div>
        </li>
        <li style="margin-bottom: 15px; display: flex; align-items: start;">
            <span style="color: #7c4dff; font-weight: bold; margin-right: 10px; font-size: 1.2em;">2.</span>
            <div><b>Hash:</b> Apply a function to map the pair to a specific bucket index.</div>
        </li>
        <li style="margin-bottom: 15px; display: flex; align-items: start;">
            <span style="color: #7c4dff; font-weight: bold; margin-right: 10px; font-size: 1.2em;">3.</span>
            <div><b>Count:</b> Increment the counter for that bucket.</div>
        </li>
    </ul>

<h2 style="color: #00e5ff; margin-top: 40px; border-left: 4px solid #00e5ff; padding-left: 15px;">3. Visual Example</h2>
    
 <p>Assume <b>Min_Support = 3</b> and Hash Function: $h(x,y) = (x \cdot 10 + y) \pmod 7$.</p>

 <table style="width:100%; border-collapse: separate; border-spacing: 0; border: 1px solid #455a64; border-radius: 8px; overflow: hidden;">
        <thead style="background-color: #37474f; color: #00e5ff;">
            <tr>
                <th style="padding: 12px;">Transaction</th>
                <th style="padding: 12px;">Generated Pair</th>
                <th style="padding: 12px;">Hash Calc</th>
                <th style="padding: 12px;">Bucket Index</th>
            </tr>
        </thead>
        <tbody style="color: #b0bec5;">
            <tr style="background-color: #263238;">
                <td rowspan="3" style="padding: 12px; border-bottom: 1px solid #455a64; vertical-align: middle; font-weight: bold;">{1, 2, 3}</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64;">{1, 2}</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64;">(12) % 7</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64; color: #ff4081; font-weight: bold;">Bucket 5</td>
            </tr>
            <tr style="background-color: #263238;">
                <td style="padding: 10px; border-bottom: 1px solid #455a64;">{1, 3}</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64;">(13) % 7</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64; color: #ff4081; font-weight: bold;">Bucket 6</td>
            </tr>
            <tr style="background-color: #263238;">
                <td style="padding: 10px; border-bottom: 1px solid #455a64;">{2, 3}</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64;">(23) % 7</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64; color: #ff4081; font-weight: bold;">Bucket 2</td>
            </tr>
        </tbody>
    </table>

<div style="margin-top: 30px; padding: 20px; background-color: #212121; border-left: 5px solid #ff4081; border-radius: 0 10px 10px 0;">
        <h3 style="margin-top: 0; color: #ff4081;">✂️ The Pruning Rule</h3>
        <p>
            When creating $C_2$, we check the bucket count for every candidate pair.
        </p>
        <p style="font-size: 1.1em; font-weight: bold; color: #fff;">
            IF <code>Bucket_Count < Min_Support</code> THEN Discard Pair.
        </p>
        <p style="font-size: 0.9em; color: #90caf9;">
            <i>Why? Because even if items A and B are individually frequent, if their combined bucket count is low, they mathematically <b>cannot</b> be frequent together.</i>
        </p>
    </div>

<h2 style="color: #00e5ff; margin-top: 40px; border-left: 4px solid #00e5ff; padding-left: 15px;">4. The "Collision" Catch</h2>
    <p>
        Sometimes, two infrequent pairs land in the same bucket, making the bucket count artificially high (False Positive).
    </p>
    
<div style="display: flex; gap: 15px; margin-top: 15px;">
        <div style="flex: 1; background-color: #1b5e20; padding: 15px; border-radius: 8px; color: #a5d6a7;">
            <b>Low Bucket Count:</b><br>
            100% Safe to Prune. <br>The pair is definitely infrequent.
        </div>
        <div style="flex: 1; background-color: #b71c1c; padding: 15px; border-radius: 8px; color: #ffcdd2;">
            <b>High Bucket Count:</b><br>
            Might be frequent, or might be a collision.<br> We must keep it and check later.
        </div>
    </div>

</div>
